In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import svm

Carregando o conjunto de dados e exibindo os 10 primeiros

In [2]:
dados = pd.read_csv("telecom_churn.csv")
dados.head(10)

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.70,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.70,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
5,0,118,0,0,0.00,0,223.4,98,57.0,11.03,6.3
6,0,121,1,1,2.03,3,218.2,88,87.3,17.43,7.5
7,0,147,0,0,0.00,0,157.0,79,36.0,5.16,7.1
8,0,117,1,0,0.19,1,184.5,97,63.9,17.58,8.7
9,0,141,0,1,3.02,0,258.6,84,93.2,11.10,11.2


Há dados faltantes?

In [3]:
dados.isna().sum()

Churn              0
AccountWeeks       0
ContractRenewal    0
DataPlan           0
DataUsage          0
CustServCalls      0
DayMins            0
DayCalls           0
MonthlyCharge      0
OverageFee         0
RoamMins           0
dtype: int64

Tipos de dados

In [4]:
dados.dtypes

Churn                int64
AccountWeeks         int64
ContractRenewal      int64
DataPlan             int64
DataUsage          float64
CustServCalls        int64
DayMins            float64
DayCalls             int64
MonthlyCharge      float64
OverageFee         float64
RoamMins           float64
dtype: object

Identificar a correlação entre as variáveis

In [5]:
dados.corr()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
Churn,1.000000,0.016541,-0.259852,-0.102148,-0.087195,0.208750,0.205151,0.018459,0.072313,0.092812,0.068239
AccountWeeks,0.016541,1.000000,-0.024735,0.002918,0.014391,-0.003796,0.006216,0.038470,0.012581,-0.006749,0.009514
ContractRenewal,-0.259852,-0.024735,1.000000,-0.006006,-0.019223,0.024522,-0.049396,-0.003755,-0.047291,-0.019105,-0.045871
DataPlan,-0.102148,0.002918,-0.006006,1.000000,0.945982,-0.017824,-0.001684,-0.011086,0.737490,0.021526,-0.001318
DataUsage,-0.087195,0.014391,-0.019223,0.945982,1.000000,-0.021723,0.003176,-0.007962,0.781660,0.019637,0.162746
CustServCalls,0.208750,-0.003796,0.024522,-0.017824,-0.021723,1.000000,-0.013423,-0.018942,-0.028017,-0.012964,-0.009640
DayMins,0.205151,0.006216,-0.049396,-0.001684,0.003176,-0.013423,1.000000,0.006750,0.567968,0.007038,-0.010155
DayCalls,0.018459,0.038470,-0.003755,-0.011086,-0.007962,-0.018942,0.006750,1.000000,-0.007963,-0.021449,0.021565
MonthlyCharge,0.072313,0.012581,-0.047291,0.737490,0.781660,-0.028017,0.567968,-0.007963,1.000000,0.281766,0.117433
OverageFee,0.092812,-0.006749,-0.019105,0.021526,0.019637,-0.012964,0.007038,-0.021449,0.281766,1.000000,-0.011023


Pela matriz de correlação acima notamos que as variáveis CustServCalls ,DayMins e ContractRenewal são mais relevantes

Verificando como os dados estão distribuídos

In [6]:
num_true = len(dados.loc[dados['Churn'] == 0])
num_false = len(dados.loc[dados['Churn'] == 1])
print(f'Número de casos verdadeiros: {num_true}, ({num_true / (num_true + num_false) * 100:.2f}%)')
print(f'Número de casos falsos (não churn): {num_false}, ({num_false / (num_true + num_false) * 100:.2f}%)')

Número de casos verdadeiros: 2850, (85.51%)
Número de casos falsos (não churn): 483, (14.49%)


Separando os dados em treino e teste

In [7]:
# Seleção de variáveis preditoras ou independentes
atributos = ['CustServCalls', 'DayMins', 'ContractRenewal']

In [8]:
# Variável alvo, dependente, a ser prevista
atrib_prev = ['Churn']

In [9]:
# Criando os objetos
X = dados[atributos].values
y = dados[atrib_prev].values

In [10]:
# Exibindo o objeto x
print(X)

[[  1.  265.1   1. ]
 [  1.  161.6   1. ]
 [  0.  243.4   1. ]
 ...
 [  2.  180.8   1. ]
 [  2.  213.8   0. ]
 [  0.  234.4   1. ]]


In [11]:
# Exibindo algumas linhas do objeto y
print(y[:25])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]]


Criando os dados de treino e teste<br>
30% para teste e 70% para treino

In [12]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [13]:
# Exibindo os resultados
print(f'{len(X_treino) / len(dados.index) * 100:.2f}% nos dados de treino.')
print(f'{len(X_teste) / len(dados.index) * 100:.2f}% nos dados de teste.')

70.00% nos dados de treino.
30.00% nos dados de teste.


### Construindo o modelo com RANDOM FOREST

In [14]:
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_treino, y_treino.ravel())

RandomForestClassifier(random_state=42)

In [15]:
# Verificando os dados de treino
rf_predict_train = random_forest.predict(X_treino)
print(f'Exatidão(Accuracy): {accuracy_score(y_treino, rf_predict_train):.4f}')

Exatidão(Accuracy): 0.9880


In [16]:
# Verificando a exatidão nos dados de teste
rf_predict_test = random_forest.predict(X_teste)
print(f'Exatidão (Accuracy): {accuracy_score(y_teste, rf_predict_test):.4f}')

Exatidão (Accuracy): 0.8490


Métricas

In [17]:
# Matriz de confusão
print('Matriz de Confusão:')
print(f'{confusion_matrix(y_teste, rf_predict_test, labels=[1, 0])}')

Matriz de Confusão:
[[ 63  80]
 [ 71 786]]


A matriz de confusão ajuda a avaliar a precisão do modelo de classificação. Neste caso, o modelo teve um número significativamente maior de verdadeiros positivos (786) em comparação com os outros valores na matriz. Isso é positivo porque, em um problema de previsão de churn, é importante identificar corretamente os casos de clientes que estão realmente propensos a cancelar o serviço (churn), o que é representado pelos verdadeiros positivos.

No entanto, também há um número considerável de falsos positivos (80), o que significa que o modelo às vezes prevê erroneamente que um cliente vai cancelar quando na verdade não vai. Também existem falsos negativos (71), o que indica que o modelo não conseguiu identificar alguns clientes que acabaram cancelando.

In [18]:
print('Classification Report')
print(f'{classification_report(y_teste, rf_predict_test, labels=[1, 0])}')

Classification Report
              precision    recall  f1-score   support

           1       0.47      0.44      0.45       143
           0       0.91      0.92      0.91       857

    accuracy                           0.85      1000
   macro avg       0.69      0.68      0.68      1000
weighted avg       0.85      0.85      0.85      1000



Analisando o relatório acima:
<p>Precision: para a classe 1 (1 = propensos ao churn), a precisão é de 0.47, o que significa que 47% das previsões positivas feitas pelo modelo são verdadeiras.</p>
<p>Recall: para a classe 1, o recall é de 0.44, o que indica que o modelo está capturando 44% dos casos reais de clientes propensos ao churn.
<p>f1-score: o F1-Score é uma média harmônica ponderada entre precisão e recall. Ele fornece um equilíbrio entre essas duas métricas. Para a classe 1, o F1-score é de 0.45.</p>
<p>Support: o suporte é o número de casos reais de cada classe no conjunto de dados. Para a classe 1, há 143 casos de clientes propensos ao churn e, para a classe 0, há 857 casos de clientes não propensos ao churn.</p>
<p>Accuracy: a acurácia geral do modelo é de 0.85, o que indica que ele está correto em 85% das previsões.</p>

### Modelo com GradientBoostingClassifier

Construindo o modelo

In [19]:
grad_boost = GradientBoostingClassifier(n_estimators=100, random_state=42)
grad_boost.fit(X_treino, y_treino)

c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(random_state=42)

Previsões nos dados de teste

In [20]:
grad_test = grad_boost.predict(X_teste)

Avaliando o desempenho do modelo com métricas de classificação.

In [21]:
print(f'Exatidão (Accuracy): {accuracy_score(y_teste, grad_test):.2f}')
print('')
print(f'Classification Report')
print(f'{classification_report(y_teste, grad_test, labels=[1, 0])}')

Exatidão (Accuracy): 0.88

Classification Report
              precision    recall  f1-score   support

           1       0.65      0.33      0.44       143
           0       0.90      0.97      0.93       857

    accuracy                           0.88      1000
   macro avg       0.77      0.65      0.68      1000
weighted avg       0.86      0.88      0.86      1000



Comparando esses dados do relatório com os do Random Forest, notamos que há uma certa melhora no desempenho usando o algoritmo GradientBoostingClassifier, pois o mesmo tem 88% de acurácia geral, contra 85% do Random Forest e também a precisão de acertos é 65%.

### Modelo com SVM

Criação do modelo

In [22]:
svm = svm.SVC(C=1.0, kernel='linear')

Treinar o modelo

In [23]:
svm.fit(X_treino, y_treino)

c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

Previsões com o modelo

In [24]:
previsoes = svm.predict(X_teste)

Avaliando o modelo

In [25]:
acuracia = np.mean(previsoes == y_teste)
print(f'Acurácia: {acuracia * 100:.2f}%')

Acurácia: 85.70%


### Qual o modelo que teve melhor desempenho para classificar churn?
<p>Vamos compara a acurácia dos modelos aqui treinados:</p>
<ul>
<li>Random Forest: teve acurárica de 85% nas previsões</li>
<li>Gradient Boosting: acurácia de 88%</li>
<li>SVM: acurácia de 85,70%</li>
</ul>
<p>Com isso podemos ver que o modelo que melhor teve desempenho foi o Gradient Boosting Classifier, cerca de 3% maior do que as outras previsões.</p>
<p>3% pode parecer uma diferença pequena, mas em um contexto de negócios pode representar uma economia significativa de dinheiro.</p>
<p>Por exemplo, se uma empresa tem 10.000 clientes, uma precisão de 88% significaria que ela poderia identificar 2.200 clientes que estão prestes a cancelar seus serviços, 200 a mais do que se usasse o random forest classifier ou o SVM.</p>